In [ ]:
%load_ext watermark


In [ ]:
import os

from IPython.display import display
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = True
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2025-02-19-cpp-bench-speed-pico"
)
teeplot_subdir


## Prep Data


In [ ]:
df = pl.concat(
    [
        pl.read_csv("https://osf.io/mtguz/download"),
    ],
)
print(df["algo_name"].unique())
df = (
    df.cast(
        {
            "memory_bytes": pl.Int32,
            "num_items": pl.Int32,
            "num_sites": pl.Int32,
            "duration_s": pl.Float64,
            "replicate": pl.Int32,
        },
    )
    .with_columns(
        duration_per_item_ns=(
            pl.col("duration_s") * 1_000_000_000 / pl.col("num_items")
        )
    )
    .with_columns(
        algorithm=pl.col("algo_name").map_elements(
            {
                "dstream.circular_algo": "control: ringbuf",
                "control_throwaway_algo": "control: discard",
                "dstream.steady_algo": "dstream steady",
                # "dstream.stretched_algo": "dstream stretched",
                # "dstream.tilted_algo": "dstream tilted",
                "dstream_steady_algo": "dstream steady",
                "dstream_tilted_algo": "dstream tilted",
                "dstream.compressing_algo": "gunther steady",
                "doubling_steady_algo": "naive steady",
                "doubling_tilted_algo": "naive tilted",
                "zhao_tilted_algo": "zhao tilted",
                "zhao_steady_algo": "zhao steady",
            }.__getitem__,
            return_dtype=str,
        )
    )
    .with_columns(pl.col("data_type").alias("data type"))
)

display(df.describe()), display(df.head()), display(df.tail());


## Plot


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with sns.plotting_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df,
            x="num_sites",
            y="duration_per_item_ns",
            col="num_items",
            row="data type",
            hue="algorithm",
            style="algorithm",
            palette="muted",
            kind="line",
            aspect=1.5,
            markers=["^", "X"],
            ms=8,
            height=2,
            teeplot_outattrs=rc,
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=True),
        ) as g:
            g.set(xscale="log", yscale="log")
            sns.move_legend(
                g,
                "lower center",
                bbox_to_anchor=(0.35, 1),
                ncol=2,
                title=None,
                frameon=False,
            )
            g.tight_layout()


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with sns.plotting_context(rc=rc):
        for num_sites in [64, 256, 1024, 4096]:
            with tp.teed(
                sns.relplot,
                data=df.filter(
                    pl.col("algorithm").is_in(
                        [
                            "control: ringbuf",
                            "control: discard",
                            "dstream tilted",
                            "gunther steady",
                            "naive tilted",
                            "zhao tilted",
                        ],
                    ),
                    pl.col("num_sites") == num_sites,
                ),
                x="num_items",
                y="duration_per_item_ns",
                col="data type",
                col_wrap=2,
                col_order=[
                    "bit",
                    "byte",
                    "word",
                    "double word",
                ],
                hue="algorithm",
                hue_order=[
                    "control: ringbuf",
                    "control: discard",
                    "dstream tilted",
                    "gunther steady",
                    "naive tilted",
                    "zhao tilted",
                ],
                style="algorithm",
                palette="muted",
                kind="line",
                aspect=1.5,
                markers=["^", "X", "P", "d", "*", "v"],
                ms=8,
                height=2,
                teeplot_subdir=teeplot_subdir,
                facet_kws=dict(margin_titles=False),
                teeplot_outattrs={
                    "num_sites": num_sites,
                    **rc,
                },
            ) as g:
                g.set(xscale="log", yscale="log")
                sns.move_legend(
                    g,
                    "upper center",
                    bbox_to_anchor=(0.42, 1.12),
                    ncol=3,
                    title=None,
                    frameon=False,
                )
                g.tight_layout()


In [ ]:
for rc in [{}, {"font.family": "serif"}]:
    with sns.plotting_context(rc=rc):
        with tp.teed(
            sns.relplot,
            data=df.filter(
                pl.col("algorithm").is_in(
                    [
                        "control: ringbuf",
                        "control: discard",
                        "dstream tilted",
                        "gunther steady",
                        "naive tilted",
                        "zhao tilted",
                    ],
                ),
                pl.col("num_items") == 100_000,
            ),
            x="num_sites",
            y="duration_per_item_ns",
            col="data type",
            col_wrap=2,
            col_order=[
                "bit",
                "byte",
                "word",
                "double word",
            ],
            hue="algorithm",
            hue_order=[
                "control: ringbuf",
                "control: discard",
                "dstream tilted",
                "gunther steady",
                "naive tilted",
                "zhao tilted",
            ],
            style="algorithm",
            palette="muted",
            kind="line",
            aspect=1.5,
            markers=["^", "X"],
            ms=8,
            height=2,
            teeplot_outattrs=rc,
            teeplot_subdir=teeplot_subdir,
            facet_kws=dict(margin_titles=False),
        ) as g:
            g.set(xscale="log", yscale="log")
            sns.move_legend(
                g,
                "upper center",
                bbox_to_anchor=(0.42, 1.12),
                ncol=3,
                title=None,
                frameon=False,
            )
            for ax in g.axes.flat:
                ax.set_xticks([64, 256, 1024, 4096])
                ax.set_xticklabels(["64", "256", "1024", "4096"])
            g.tight_layout()


In [ ]:
# # 1) Compute mean duration per group x algo
# df_agg = df.group_by(
#     ["compiler", "num_items", "num_sites", "replicate", "algo_name", "data type"]
# ).agg(mean_duration_s=pl.col("duration_s").mean())

# # 2) Compute mean duration *just for* "naive_steady_algo"
# df_naive = (
#     df.filter(pl.col("algo_name") == "naive_steady_algo")
#     .group_by(["compiler", "num_items", "num_sites", "replicate", "data type"])
#     .agg(naive_mean_duration_s=pl.col("duration_s").mean())
# )


# # 3) Join the two on compiler,num_items,num_sites,replicate
# df_speedup = df_agg.join(
#     df_naive,
#     on=["compiler", "num_items", "num_sites", "replicate", "data type"],
#     how="left",
# ).with_columns(
#     (pl.lit(1_000_000_000) * pl.col("mean_duration_s") / pl.col("num_items"))
#     .alias("ns_per_op"),
#     # 4) Speedup = ratio vs. naive_steady_algo
#     (pl.col("naive_mean_duration_s") / pl.col("mean_duration_s")).alias(
#         "speedup"
#     )
# )

# # Display tabular results
# with pd.option_context("display.max_columns", None):
#     with pd.option_context("display.max_rows", None):
#         display(
#             df_speedup.filter(pl.col("num_items") == 1_000_000)
#             .group_by(["num_sites", "algo_name", "data type"])
#             .agg(
#                 speedup_mean=pl.col("speedup").mean(),
#                 speedup_std=pl.col("speedup").std(),
#                 ns_per_op_mean=pl.col("ns_per_op").mean(),
#                 ns_per_op_std=pl.col("ns_per_op").std(),
#             )
#             .sort(["algo_name", "num_sites"])
#             .to_pandas()
#         )

# # Display tabular results
# with pd.option_context("display.max_columns", None):
#     with pd.option_context("display.max_rows", None):
#         display(
#             df_speedup.filter(pl.col("num_items") == 10_000)
#             .group_by(["num_sites", "algo_name", "data type"])
#             .agg(
#                 speedup_mean=pl.col("speedup").mean(),
#                 speedup_std=pl.col("speedup").std(),
#                 ns_per_op_mean=pl.col("ns_per_op").mean(),
#                 ns_per_op_std=pl.col("ns_per_op").std(),
#             )
#             .sort(["algo_name", "num_sites", "data type"])
#             .to_pandas()
#         )

# # Now plot speedup on a log scale
# with tp.teed(
#     sns.catplot,
#     data=df_speedup.cast({"num_sites": str}).filter(
#         pl.col("algo_name") != "naive_steady_algo"
#     ),
#     x="num_sites",
#     y="speedup",
#     order=list(map(str, sorted(df["num_sites"].unique()))),
#     hue="algo_name",
#     col="compiler",
#     row="data type",
#     kind="bar",
#     aspect=1.4,
#     height=2.5,
#     teeplot_subdir=teeplot_subdir,
# ) as g:
#     g.set(yscale="log", ylim=(1, None))
#     for ax in g.axes.flat:
#         ax.bar_label(
#             ax.containers[1],
#             fmt="$%d\\times$",
#             label_type="center",
#             rotation=90,
#         )
